In [58]:
import xlrd
import pandas as pd
import numpy as np
import re

In [59]:
book = xlrd.open_workbook('GSAF5.xls',formatting_info=True)
sheet = book.sheet_by_index(0)

In [60]:
colors = []
for item in range(5717):
    xfx = sheet.cell_xf_index(item, 4)
    xf = book.xf_list[xfx]
    bgx = xf.background.pattern_colour_index
    colors.append(bgx)

In [3]:
set(colors)

NameError: name 'colors' is not defined

In [62]:
colorlist={8:'index',
60:'Unprovoked Incidents',
47:'Provoked Incidents',
42:'Incidents Involving Boats',
43:'Air / Sea Disasters',
44:'Questionable Incidents',
64:'blank',
22:'Steve Irwin'}

In [63]:
attack_type = [colorlist[x] for x in colors[1:]]

In [64]:
df= pd.read_excel('GSAF5.xls')

In [65]:
df['Attack_Type'] = attack_type
df['Activity'] = df['Activity'].str.lower()
df['Activity'] = df['Activity'].str.replace('-','')
df['Species '] = df['Species '].str.lower()
df['Fatal'] = df['Injury'].str.contains('FATAL')
df['Country'] = df['Country'].str.upper()

In [1]:
def year(x):
    res = re.search(r'([0-9]{4})',str(x))
    if res is None:
        return None
    else: 
        return int(res.group())

def day(x):
    res = re.search(r'(^|\b)(([0-9]{2}))(?=-)',str(x))
    if res is None:
        return None
    else:
        return int(res.group())
    
def month(x):
    res = re.search(r'(?!-)(([A-Za-z]{3}))(?=-)',str(x))
    if res is None:
        return None
    else:
        return str(res.group())

df['year'] = df['Date'].map(year)
df['month'] = df['Date'].map(month)
df['day'] = df['Date'].map(day)

NameError: name 'df' is not defined

In [67]:
def types(x):
    if re.search(r'\bscuba\b',str(x)):
        x='scuba_diving'
        return x
    elif re.search(r"\bspearfishing\b",str(x)) and not(re.search(r'\bscuba\b',str(x))):
        x = 'spear_fishing'
        return x
    elif re.search(r"\bswimming\b",str(x)):
        x = 'swimming'
        return x
    elif re.search(r"\bstanding\b",str(x)):
        x = 'standing'
        return x
    elif re.search("bodysurfing",str(x)) or (re.search("body surfing",str(x))):
        x = 'body_surfing'
        return x
    elif re.search("bodyboarding",str(x)) or (re.search("body-boarding",str(x)))or (re.search("body boarding",str(x))):
        x = 'body_boarding'
        return x
    elif re.search(r"\bsurfing\b",str(x)) or (re.search("surfboard",str(x))):
        x = 'surfing'
        return x
    elif re.search("surf-skiing",str(x)) or (re.search("surf skiing",str(x))) or (re.search("surfskiing",str(x))):
        x = 'surf_skiing'
        return x
    elif re.search("pearl diving",str(x)):
        x = 'pearl_diving'
        return x
    elif re.search(r"\bdiving\b",str(x)):
        x = 'diving'
        return x
    elif re.search(r"\bspear\b",str(x)):
        x = 'spear_fishing'
        return x
    elif re.search(r"\bbathing\b",str(x)):
        x = 'bathing'
        return x
    elif re.search(r"\bfishing\b",str(x)):
        x = 'fishing'
        return x
    elif re.search(r"\bfreediving\b",str(x)) or (re.search("free diving",str(x))):
        x = 'free_diving'
        return x
    elif re.search("boogie",str(x)):
        x = 'boogie_boarding'
        return x
    elif re.search("capsized",str(x)) or (re.search("sank",str(x))) or (re.search("went down",str(x))) \
    or (re.search("disaster",str(x))) or (re.search("crash",str(x))) or (re.search("wreck",str(x))):
        x = 'sea_disaster'
        return x
    elif re.search(r"\bwading\b",str(x)):
        x = 'wading'
        return x
    else: return x

df['Activity'] = df['Activity'].map(types)

In [71]:
df = df.fillna('')
df.head()
# df.to_csv('./sharks_cleaned.csv',index=False)

,Case Number,Date,Country,Area,Location,Activity,Name,Sex,Age,Injury,Time,Species,Investigator or Source,Attack_Type,Fatal,year,month,day
0,ND.0001,1845-1853,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",swimming,male,M,15,"FATAL. ""Shark bit him in half, carrying away t...",,,S.W. Baker,Unprovoked Incidents,True,1845,,
1,ND.0002,1883-1889,PANAMA,,"Panama Bay 8ºN, 79ºW",,Jules Patterson,M,,FATAL,,,"The Sun, 10/20/1938",Unprovoked Incidents,True,1883,,
2,ND.0003,1900-1905,USA,North Carolina,Ocracoke Inlet,swimming,Coast Guard personnel,M,,FATAL,,,"F. Schwartz, p.23; C. Creswell, GSAF",Unprovoked Incidents,True,1900,,
3,ND.0004,Before 1903,AUSTRALIA,Western Australia,,pearl_diving,Ahmun,M,,FATAL,,,"H. Taunton; N. Bartlett, pp. 233-234",Unprovoked Incidents,True,1903,,
4,ND.0005,Before 1903,AUSTRALIA,Western Australia,Roebuck Bay,diving,male,M,,FATAL,,,"H. Taunton; N. Bartlett, p. 234",Unprovoked Incidents,True,1903,,


In [139]:
def fix_locs(x):
    if str(x.Location)!=None:
        if 'Klamath River' in str(x.Location):
            return (41.5444542,-124.0705673)
        elif str(x.Location)=='South Moss Beach, Spanish Bay, Monterey Peninsula':
            return (36.610643, -121.949255)
        elif str(x.Location)=='Davenport Landing':
            return (37.023281, -122.215310)
        elif str(x.Location)=='Capistrano, Orange County':
            return (33.458935, -117.682083)
        elif str(x.Location)=='North Farallon Island, Farallon Islands':
            return (37.767277, -123.098960)
        elif str(x.Location)=='West Cove, Catalina Island':
            return (33.472051, -118.604834)
        elif str(x.Name)=='Norman Piexotto':
            return (37.733333, -122.256667)
        elif str(x.Location)=='Pigeon Point':
            return (37.183001, -122.394740)
        elif str(x.Location)=='Usal Creek, Bear Harbor, Mendocino County':
            return (39.913219, -123.938730)
        elif str(x.Name)=='David Bull':
            return (33.375996, -117.569556)
        else:
            return x.coords
    return x.coords
        
def extract_lat(x):
    if x!=None:
        return x[0]
    else:
        return None
    
def extract_lon(x):
    if x!=None:
        return x[1]
    else:
        return None

In [140]:
result["coords"] = result.apply(fix_locs,axis=1)
result["latitude"] = result["coords"].map(extract_lat)
result["longitude"] = result["coords"].map(extract_lon)